# Week 11 - Introduction to Modeling, part 2

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, GridSearchCV, RandomizedSearchCV, LeaveOneOut, RepeatedKFold
from sklearn.metrics import make_scorer

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
# Import data

from google.colab import files

df = files.upload()

Saving NetworkSecurity_Cleaned.csv to NetworkSecurity_Cleaned.csv


In [3]:
df = pd.read_csv("NetworkSecurity_Cleaned.csv")

pd.set_option('display.max_columns', None)

df.head()

,Timestamp,Source IP Address,Destination IP Address,Source Port,Destination Port,Protocol,Packet Length,Packet Type,Traffic Type,Payload Data,Malware Indicators,Anomaly Scores,Alerts/Warnings,Attack Type,Attack Signature,Action Taken,Severity Level,User Information,Device Information,Network Segment,Geo-location Data,Proxy Information,Firewall Logs,IDS/IPS Alerts,Log Source
0,2023-05-30 06:33:58,103.216.15.12,84.9.164.252,31225,17616,ICMP,503,Data,HTTP,Qui natus odio asperiores nam. Optio nobis ius...,IoC Detected,28.67,Alert Triggered,Malware,Known Pattern B,Logged,Low,Reyansh Dugal,Mozilla/5.0 (compatible; MSIE 8.0; Windows NT ...,Segment A,"Jamshedpur, Sikkim",150.9.97.135,Log Data,Alert Data,Server
1,2020-08-26 07:08:30,78.199.217.198,66.191.137.154,17245,48166,ICMP,1174,Data,HTTP,Aperiam quos modi officiis veritatis rem. Omni...,IoC Detected,51.50,Alert Triggered,Malware,Known Pattern A,Blocked,Low,Sumer Rana,Mozilla/5.0 (compatible; MSIE 8.0; Windows NT ...,Segment B,"Bilaspur, Nagaland",39.123.165.122,Log Data,Alert Data,Firewall
2,2022-11-13 08:23:25,63.79.210.48,198.219.82.17,16811,53600,UDP,306,Control,HTTP,Perferendis sapiente vitae soluta. Hic delectu...,IoC Detected,87.42,Alert Triggered,DDoS,Known Pattern B,Ignored,Low,Himmat Karpe,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,Segment C,"Bokaro, Rajasthan",114.133.48.179,Log Data,Alert Data,Firewall
3,2023-07-02 10:38:46,163.42.196.10,101.228.192.255,20018,32534,UDP,385,Data,HTTP,Totam maxime beatae expedita explicabo porro l...,IoC Detected,15.79,Alert Triggered,Malware,Known Pattern B,Blocked,Medium,Fateh Kibe,Mozilla/5.0 (Macintosh; PPC Mac OS X 10_11_5; ...,Segment B,"Jaunpur, Rajasthan",39.123.165.122,Log Data,Alert Data,Firewall
4,2023-07-16 13:11:07,71.166.185.76,189.243.174.238,6131,26646,TCP,1462,Data,DNS,Odit nesciunt dolorem nisi iste iusto. Animi v...,IoC Detected,0.52,Alert Triggered,DDoS,Known Pattern B,Blocked,Low,Dhanush Chad,Mozilla/5.0 (compatible; MSIE 5.0; Windows NT ...,Segment C,"Anantapur, Tripura",149.6.110.119,Log Data,Alert Data,Firewall


In [4]:
df_copy = df.copy()

cat_var = df_copy[["Traffic Type", "Protocol", "Packet Type", "Attack Type", "Attack Signature", "Action Taken", "Network Segment", "Log Source",
                   ]]

dummy_df = pd.get_dummies(cat_var)

df_copy = df_copy.drop(columns=["Timestamp", "Traffic Type", "Protocol", "Packet Type", "Attack Type", "Attack Signature",
                                "Action Taken", "Network Segment", "Log Source", 'Source IP Address',	'Destination IP Address',
                                'Source Port',	'Destination Port', 'Payload Data', 'Device Information', 'User Information',
                                'Geo-location Data',	'Proxy Information'])

df_enc = pd.concat([df_copy, dummy_df], axis=1)


In [5]:
level = {
    "Low" : 1,
    "Medium" : 2,
    "High" : 3
}

df_enc['Severity Level'] = df_enc['Severity Level'].map(level)
df_enc['Severity Level']

,Severity Level
0,1
1,1
2,1
3,2
4,1
...,...
39995,2
39996,3
39997,1
39998,1


In [6]:
pd.set_option('display.max_columns', None)
df_enc.head()

,Packet Length,Malware Indicators,Anomaly Scores,Alerts/Warnings,Severity Level,Firewall Logs,IDS/IPS Alerts,Traffic Type_DNS,Traffic Type_FTP,Traffic Type_HTTP,Protocol_ICMP,Protocol_TCP,Protocol_UDP,Packet Type_Control,Packet Type_Data,Attack Type_DDoS,Attack Type_Intrusion,Attack Type_Malware,Attack Signature_Known Pattern A,Attack Signature_Known Pattern B,Action Taken_Blocked,Action Taken_Ignored,Action Taken_Logged,Network Segment_Segment A,Network Segment_Segment B,Network Segment_Segment C,Log Source_Firewall,Log Source_Server
0,503,IoC Detected,28.67,Alert Triggered,1,Log Data,Alert Data,False,False,True,True,False,False,False,True,False,False,True,False,True,False,False,True,True,False,False,False,True
1,1174,IoC Detected,51.50,Alert Triggered,1,Log Data,Alert Data,False,False,True,True,False,False,False,True,False,False,True,True,False,True,False,False,False,True,False,True,False
2,306,IoC Detected,87.42,Alert Triggered,1,Log Data,Alert Data,False,False,True,False,False,True,True,False,True,False,False,False,True,False,True,False,False,False,True,True,False
3,385,IoC Detected,15.79,Alert Triggered,2,Log Data,Alert Data,False,False,True,False,False,True,False,True,False,False,True,False,True,True,False,False,False,True,False,True,False
4,1462,IoC Detected,0.52,Alert Triggered,1,Log Data,Alert Data,True,False,False,False,True,False,False,True,True,False,False,False,True,True,False,False,False,False,True,True,False


In [7]:
# Logistic Regression

# Use important features only captured from PCA (HW 9.2)
X = df_enc[["Packet Length", 'Anomaly Scores', 'Traffic Type_DNS', 'Traffic Type_FTP',
       'Traffic Type_HTTP', 'Protocol_ICMP', 'Protocol_TCP', 'Protocol_UDP', 'Packet Type_Control', 'Packet Type_Data',
            'Network Segment_Segment A', 'Network Segment_Segment B', 'Network Segment_Segment C', 'Log Source_Firewall', 'Log Source_Server']]

y = df_enc['Severity Level']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,test_size=0.2)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model_LR = LogisticRegression(max_iter=2000)

model_LR.fit(X_train_scaled, y_train)

y_pred = model_LR.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.333625

Classification Report:
               precision    recall  f1-score   support

           1       0.33      0.23      0.27      2673
           2       0.35      0.35      0.35      2729
           3       0.32      0.42      0.37      2598

    accuracy                           0.33      8000
   macro avg       0.33      0.33      0.33      8000
weighted avg       0.33      0.33      0.33      8000


Confusion Matrix:
 [[ 603  917 1153]
 [ 618  967 1144]
 [ 624  875 1099]]


In [8]:
# Random Forest Classifier

param_grid = {
    'n_estimators' : [10,25,40],
    'max_depth' : [15, 20, 25],
    'max_features' : [0.8, 0.9],
    'bootstrap' : [True]
}

model_RFC = RandomForestClassifier(random_state=42)

randsearchcv = RandomizedSearchCV(model_RFC, cv=5, random_state=42, param_distributions=param_grid, n_iter=5, n_jobs=-1)

randsearchcv.fit(X_train_scaled, y_train)

print("Best Parameters:", randsearchcv.best_params_)
print("Best Cross-Val Score:", randsearchcv.best_score_)

# Best Parameters: {'n_estimators': 100, 'max_features': 0.9, 'max_depth': 20, 'bootstrap': True}
# Best Cross-Val Score: 0.33971875
# Not a good model as accuracy is only 33%

Best Parameters: {'n_estimators': 10, 'max_features': 0.8, 'max_depth': 15, 'bootstrap': True}
Best Cross-Val Score: 0.33971875


In [9]:
param_grid2 = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [10, 20, 30, None],
    'max_features': ['sqrt', 0.5, 0.8, 1.0],
    'bootstrap': [True]
}

model_rfr2 = RandomForestRegressor(random_state=42)

cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)

rand_search = RandomizedSearchCV(
    estimator=model_rfr2,
    param_distributions=param_grid2,
    n_iter=5,
    scoring='neg_mean_squared_error',
    cv=cv,
    n_jobs=-1,
    random_state=42
)

rand_search.fit(X_train, y_train)

best_model = rand_search.best_estimator_
rmse = np.sqrt(-rand_search.best_score_)

print("Best Parameters:", rand_search.best_params_)
print("Best RMSE:", rmse)

# Best Parameters: {'n_estimators': 100, 'max_features': 0.5, 'max_depth': 10, 'bootstrap': True}
# Best RMSE: 0.8170150582361623
# This is not a good model either because the 'y range' of this model is 1-3. So each prediction if off by 0.82 units, and that is a large margin.

Best Parameters: {'n_estimators': 100, 'max_features': 0.5, 'max_depth': 10, 'bootstrap': True}
Best RMSE: 0.8170150582361623
